<a href="https://colab.research.google.com/github/nidhaloff/indians-diabetes-/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import torch
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from sklearn.preprocessing import StandardScaler

# ignore deprication warnings
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

# set device to gpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device => ", device)

%matplotlib inline

# run keras model on google GPU
K.tensorflow_backend._get_available_gpus()
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
import warnings


device =>  cuda


In [82]:
dataset = pd.read_csv('/content/diabet_dataset.txt')
X = dataset.iloc[:, 0:-1]
y = dataset.iloc[:, -1]
print(X.shape, y.shape)
print(dataset.head(5))
print(X.head(5))
print(y.head(5))

(767, 8) (767,)
   6  148  72  35    0  33.6  0.627  50  1
0  1   85  66  29    0  26.6  0.351  31  0
1  8  183  64   0    0  23.3  0.672  32  1
2  1   89  66  23   94  28.1  0.167  21  0
3  0  137  40  35  168  43.1  2.288  33  1
4  5  116  74   0    0  25.6  0.201  30  0
   6  148  72  35    0  33.6  0.627  50
0  1   85  66  29    0  26.6  0.351  31
1  8  183  64   0    0  23.3  0.672  32
2  1   89  66  23   94  28.1  0.167  21
3  0  137  40  35  168  43.1  2.288  33
4  5  116  74   0    0  25.6  0.201  30
0    0
1    1
2    0
3    1
4    0
Name: 1, dtype: int64


In [83]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

y_train, y_test = y_train.to_numpy(), y_test.to_numpy()
print(X_train.shape, y_test.shape)

(613, 8) (154,)


In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [0]:
def create_model(layers, activation):
  model = Sequential()
  for i, nodes in enumerate(layers):
    if i ==0:
      model.add(Dense(nodes, input_dim=X_train.shape[1]))
      model.add(Activation(activation))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activation))
  
  model.add(Dense(1))    # no activation beyond this point
  
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

model = KerasClassifier(build_fn=create_model, verbose=0)

In [0]:
layers = [ [20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']

param_grid = dict(layers=layers, activation=activations, batch_size=[20, 40, 60], epochs=[20])
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)

In [87]:
#print(type(X_train), type(y_train))
#print(set(y_train.to_numpy()))
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [91]:
score, params = [grid_result.best_score_, grid_result.best_params_]

print("best score: ", score)
print("best params: ", params)



best score:  0.7406199023151865
best params:  {'activation': 'relu', 'batch_size': 40, 'epochs': 20, 'layers': [45, 30, 15]}


In [92]:
y_pred = grid.predict(X_test)
print(y_pred.shape, y_test.shape)

(154, 1) (154,)


In [93]:
print(type(y_pred), type(y_test))
print(y_pred[:20])
print(y_test[:20])

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1]
